## Chapter 4: Regression And Prediction: Stepwise Linear Regression- Page 238
    Contains Functions:
       Stepwise Linear Regression

In [102]:
# import modules
import math
import pylab
import random
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from sklearn.model_selection import train_test_split

### Multiple Linear Regression
    ![Model Formula](https://wikimedia.org/api/rest_v1/media/math/render/svg/704b31aa61dfc93d672f15bf02aa6d168be49643)

### Create Test Data

In [238]:
features_size=30 #number of features
def create_linear_data(n_samples=10000,f=features_size):   
    # Get regression data from scikit-learn
    x,y = datasets.make_regression(n_samples=n_samples,noise=80, n_features=f)
    x=pd.DataFrame(x)
    y=pd.DataFrame(y,columns=['Y'])
    df = pd.concat([x, y],axis = 1)
    return df

df = create_linear_data()
df.head()


0         1         2         3         4         5         6  \
0 -1.150228 -0.647258 -0.258421  0.749926  0.186117  0.695557 -1.080732   
1  1.193624  0.637780 -0.222977  0.796864  0.022765 -0.095462 -0.487500   
2 -2.377882  0.757898  0.199256 -1.187256  0.185342 -0.091670  0.302742   
3  0.913615 -0.132294 -0.739062 -1.600399 -0.183899 -0.361266 -0.459224   
4  1.836089 -0.944510 -2.638461  0.393566  1.713522  1.028408  1.549729   

          7         8         9  ...        21        22        23        24  \
0 -0.227240 -2.589893  0.269304  ...  0.859401  1.107162  0.041735  0.565340   
1  0.402595 -0.560716  0.632586  ...  1.689768 -0.203603  0.299804  0.495515   
2 -1.544767  0.205811 -0.841876  ... -0.514926  0.812424  1.153086 -0.204190   
3 -0.118342  0.056168  1.343363  ...  0.743626  1.974761 -0.718708 -0.619913   
4  0.685787  2.459642  0.607970  ...  0.318129  0.110768 -0.127362  0.416896   

         25        26        27        28        29           Y  
0  1.015892 -2.128823  2.024022 -0.333231 -0.204844  160.636216  
1 -2.020823 -0.243168 -0.264210  1.670538  1.882344 -190.250377  
2 -0.744905 -0.265578 -0.711802  0.085286  2.074812  -15.315213  
3  0.618423  0.793103 -0.304795 -0.805131  0.231316   37.663762  
4 -1.572623  0.925379  0.248693 -0.884124  0.853936  139.049296  

[5 rows x 31 columns]

### Split Train and Test Set

In [239]:
x = df.drop(columns=['Y'],axis=1,inplace=False)
y= df['Y']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33)
print(f'Train Data Size: {len(x_train)} Samples, Test Data Size: {len(x_test)} Samples.')

Train Data Size: 6700 Samples, Test Data Size: 3300 Samples.


### Create Linear Model Object Class

In [240]:
#create linear model object
class linear_model:
    def __init__(self):
        self._beta_l = []
        self.y_p = []
        self.features= []
        
    def __get_beta(self,x,y):
        for f in self.features:
            x_sub = np.array(x[f])
            
            n = len(x_sub)
            x_b = x_sub.mean()
            y_b = y.mean()
            nume = 0  #numerator of the formula
            denom = 0 #denominator of the formula
            for i in range(0,n):
                nume += (x_sub[i]-x_b)*(y[i]-y_b)  #summation of numerator
                denom += (x_sub[i]-x_b)**2 #summation of denominator

            beta = nume/denom  #divide
            self._beta_l.append(beta) #append beta
                               
    def fit(self,x,y):
        y= np.array(y)
        self.features = list(x.columns.values.tolist()) #get list of features
        self.__get_beta(x,y)         
        y_temp = []
        y_p = []
        for f_i,f in enumerate(self.features):
            x_sub = np.array(x[f])
            y_sub = []
            for i in range(0,len(x_sub)):
                y_i = self._beta_l[f_i]*x_sub[i]
                y_sub.append(y_i)
            y_temp.append(y_sub)
        for i in range(0,len(y)):
            _sum = 0
            for f in self.features:
                _sum+=y_temp[f_i][i]
            self.y_p.append(_sum)
                               
    def predict(self,x_test):
        y_pred = []
        test_features = list(x_test.columns.values.tolist()) #get list of features
        x_test= np.array(x_test)
        #test features should be equal to model features
        if len(test_features) == len(self.features):
            x_n = len(x_test)
            for x_i in range(x_n):
                y_i = 0
                for f_i,f in enumerate(test_features):
                    y_i += self._beta_l[f_i]*x_test[x_i][f_i]
                y_pred.append(y_i)
            
            return np.array(y_pred)
        else:
            return None
            
# Train Model                               
model = linear_model()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

### Test with R Squared

In [241]:
def get_rsquared(y,y_f):
    #compute total sum of squares
    ss_tot = 0
    y = np.array(y)
    y_f = np.array(y_f)
    y_b = y.mean()
    for i in range(0,len(y)):
        ss_tot+=(y[i]-y_b)**2
    #compute residual sum of squares
    ss_res = 0
    for i in range(0,len(y)):
        ss_res += (y[i]-y_f[i])**2
    
    return 1-(ss_res/ss_tot)

r_squared = get_rsquared(y_test,y_pred)
print(f'R Squared: {r_squared}')

R Squared: 0.8312549234762069


### Test with RMSE

In [242]:
def get_rmse(y,y_f):
    '''
        params: y = actual values, y_f = predicted values
    '''
    _se=0
    y = np.array(y)
    y_f = np.array(y_f)
    for i in range(0,len(y_f)):
        _se += (y[i]-y_f[i])**2
    rmse = math.sqrt(_se/len(y))
    return rmse

rmse = get_rmse(y_test,y_pred)
print(f'RMSE: {rmse}')

RMSE: 81.49520977166252


### Apply Stepwise Regression Method

In [251]:
# use backward method
def stepwise_regression(x_train,x_test,y_train,y_test):
    '''
        Will gradually drop features until lowest possible rmse can be attained.
    '''
    features = list(x_train.columns.values.tolist())
    n=len(features)
    rmse_l = []
    r2_l = []
    to_drop = []
    p_change_l = [0]
    model_l = []
    for s_i in range(n-1,-1,-1):
        if s_i == n-1:
            drop_n = [s_i]
        else:
            drop_n = [s_i]+to_drop
        x_train_sub = x_train.drop(columns=drop_n,inplace=False)
        x_test_sub = x_test.drop(columns=drop_n,inplace=False)
        model = linear_model()
        model.fit(x_train_sub,y_train)
        y_pred = model.predict(x_test_sub)
        _rmse = get_rmse(y_test,y_pred)
        rmse_l.append(_rmse)
        if s_i < n-1:
            p_change = (rmse_l[n-s_i-2]-rmse_l[n-s_i-1])/rmse_l[n-s_i-1]  # compute for p change
            p_change_l.append(p_change)
            
            if p_change_l[n-s_i-1] > p_change_l[n-s_i-2]: # check if percent change is higher
                if rmse_l[n-s_i-1] < rmse_l[n-s_i-2]: #check if rmse is lower
                    to_drop.append(s_i) #if all conditions passed drop the column
                    print(f'Dropping feauture: {s_i}, Percent Change: {p_change} , RMSE: {_rmse}')
    
    return to_drop
     

to_drop = stepwise_regression(x_train,x_test,y_train,y_test)
x_train_sub = x_train.drop(columns=to_drop,inplace=False)
x_test_sub = x_test.drop(columns=to_drop,inplace=False)
model = linear_model()
model.fit(x_train_sub,y_train)
y_pred = model.predict(x_test_sub)
_rmse = get_rmse(y_test,y_pred)
print(f'RMSE Final: {_rmse}')


Dropping feauture: 27, Percent Change: 0.01229246523467293 , RMSE: 81.49452213511678
Dropping feauture: 24, Percent Change: 0.2504170843492308 , RMSE: 82.20510385867695
Dropping feauture: 21, Percent Change: 0.35497813510475035 , RMSE: 82.10409251388421
Dropping feauture: 19, Percent Change: 0.0007562972163681462 , RMSE: 82.08263327409574
Dropping feauture: 15, Percent Change: 0.5186821532159513 , RMSE: 81.91703067549797
Dropping feauture: 12, Percent Change: 1.5266260703905366e-05 , RMSE: 81.91714115984522
Dropping feauture: 9, Percent Change: 0.4325919468048653 , RMSE: 81.91697358652897
Dropping feauture: 7, Percent Change: 0.0019143160112033437 , RMSE: 81.91555473753654
Dropping feauture: 4, Percent Change: 0.00017722278488857847 , RMSE: 81.90275678795679
Dropping feauture: 2, Percent Change: 0.040759705068031823 , RMSE: 81.68237268152592
RMSE Final: 81.68237268152592
